# SILVER Layer: NoteBook Transformation


#### Load pyspark functions


In [ ]:
from pyspark.sql.functions import *

#### Loading the Data into DataFrames

In [ ]:
df_customer = spark.read.format("csv").option("header","true").load("Files/DMartBronze_customers/Customers.csv")
df_Order = spark.read.format("csv").option("header","true").load("Files/DMartBronze_orders/Orders.csv")
df_Products = spark.read.format("csv").option("header","true").load("Files/DMartBronze_products/Products.csv")
df_review = spark.read.json("Files/DMartBronze_review/Review.json")
df_socialmedia = spark.read.json("Files/DMartBronze_socialmedia/SocialMedia.json")
df_weblogs = spark.read.json("Files/DMartBronze_web_logs/WebLogs.json")

## Data Cleaning and validating

In [ ]:
df_review = df_review.dropna(subset=["customer_id","product_id","rating","review_text","timestamp"])


#### Orders Data cleaning

In [ ]:
df_Order = df_Order.dropna(subset=["OrderID","OrderDate","CustomerID","ProductID","TotalAmount"])
#df_testod = df_Order.withColumn("OrderDate",to_date(to_timestamp("OrderDate")))
df_order = df_Order.withColumn("OrderDate",to_date(to_timestamp("OrderDate")))
df_order = df_order.withColumn("Quantity",col("Quantity").cast("float"))
df_order = df_order.withColumn("TotalAmount",col("TotalAmount").cast("float"))

#### Cutomers Data cleaning

In [ ]:
df_customer = df_customer.dropna(subset=["CustomerID","CustomerName","Email","Location","SignupDate"])
df_customer =df_customer.withColumn("SignupDate",to_date(col("SignupDate")))
df_customer = df_customer.withColumn("Domain",split(col("Email"),"@")[1])
df_customer= df_customer.withColumn("First_Name",split(col("CustomerName")," ")[0])
df_customer=df_customer.withColumn("Last_Name",split(col("CustomerName")," ")[1])

#### Products Data cleaning

In [ ]:
df_products = df_Products.dropna(subset=["ProductID","ProductName","Category","Stock","UnitPrice"])
df_products = df_Products.withColumn("UnitPrice",col("UnitPrice").cast("float"))
df_products =df_products.withColumn("Stock",col("Stock").cast("int"))

#### Dataframe consolidation using Join

In [70]:
df_joinedtable = df_order.\
                join(df_customer, on="CustomerID",how ="inner").\
                join(df_products,on="ProductID", how ="inner")

StatementMeta(, e11673fc-139b-4b61-be59-4f18e520316f, 72, Finished, Available, Finished)

#### Writing Data to Sliver Layer Data cleaning

In [72]:
df_joinedtable.write.mode("overwrite").parquet("Files/DMartSliver_Data/Sliver_Order_Data")
df_review.write.mode("overwrite").parquet("Files/DMartSliver_review/Review")
df_socialmedia.write.mode("overwrite").parquet("Files/DMartSliver_socialmedia/SocialMedia")
df_weblogs.write.mode("overwrite").parquet("Files/DMartSliver_web_logs/WebLogs")



StatementMeta(, e11673fc-139b-4b61-be59-4f18e520316f, 74, Finished, Available, Finished)